In [23]:
_input_path = experiment_data_dir / "evidence" / "combined_score.json"
with _input_path.open() as f:
    combined_evidence = json.load(f)
print(len(combined_evidence))

413


In [24]:
_input_path = analysis_dir / "terms_match_epigraphdb.json"
with _input_path.open() as f:
    terms_match_epigraphdb = json.load(f)
print(len(terms_match_epigraphdb))
print(terms_match_epigraphdb[0])

413
{'doi': '10.1101/19001719', 'subject_term': 'Antihypertensive Agents', 'object_term': 'Mood Disorders', 'subject_term_match': {'efo': [], 'gwas': [], 'literature_term': [{'n': {'_name': 'Antihypertensive Agents', 'name': 'Antihypertensive Agents', '_source': ['SemMedDB_VER42_2020_R', 'bioRxiv-2020-10-06', 'medRxiv-2020-10-06'], 'id': 'C0003364', '_id': 'C0003364', 'type': ['phsu']}}]}, 'object_term_match': {'efo': [], 'gwas': [], 'literature_term': []}}


----

filter evidence where the associated subject and object entities 
have been identified by epigraphdb direct search

In [33]:
# check which entries to look at
def _filter_evidence(evidence_list, subject_term_matches, object_term_matches):
    res = (
        py_.chain(evidence_list)
        .filter(
            lambda e: e["subject_id"] in subject_term_matches
            and e["object_id"] in object_term_matches
        )
        .value()
    )
    return res


for idx in range(100):
    flag = False
    print(idx)
    foo = terms_match_epigraphdb[idx]
    subject_term_matches = (
        py_.chain(foo["subject_term_match"]["gwas"]).map(lambda e: e["n"]["id"]).value()
    )
    object_term_matches = (
        py_.chain(foo["object_term_match"]["gwas"]).map(lambda e: e["n"]["id"]).value()
    )
    bar = combined_evidence[idx]["assoc_evidence"]
    evidence_types = bar.keys()
    evidence_filter = {
        _: _filter_evidence(bar[_], subject_term_matches, object_term_matches)
        for _ in evidence_types
    }
    for _ in evidence_types:
        if len(evidence_filter[_]) > 0:
            print(_)
            print(evidence_filter[_])
            flag = True
            break
    if flag:
        break

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
contradictory_directional_type2
[{'subject_id': 'ukb-d-30850_irnt', 'subject_term': 'Testosterone', 'object_id': 'ukb-d-C3_PROSTATE', 'object_term': 'Malignant neoplasm of prostate', 'meta_rel': 'MR_EVE_MR', 'effect_size': -0.0042728189633286, 'se': 0.00284892063202589, 'pval': 0.136913341653637, 'rel_data': {'b': -0.0042728189633286, 'se': 0.00284892063202589, 'method': 'Simple mean', 'nsnp': 98, 'selection': 'DF', 'pval': 0.136913341653637, 'ci_upp': 0.00131106547544215, '_source': ['MR-EvE-2021-03-10'], 'ci_low': -0.009856703402099341, 'moescore': 0.75}, 'direction': 'forward', 'idx': 10, 'mapping_score': 0.9034557629084101, 'assoc_score': 1.1760341420068288, 'mapping_data': {'subject_mapping_score': 1.0, 'object_mapping_score': 0.9034557629084101, 'subject_mapping_data': [{'ent_id': 'ukb-d-30850_irnt', 'ent_term': 'Testosterone', 'text': 'testosterone', 'similarity_score': 1.0, 'meta_node': 'Gw